ETL
=

In [51]:
#Read and uncomppresing files
import gzip
import json

#Basic data treatment
import pandas as pd
import numpy as np
import ast

#Regular expresions
import re

#Datetime functionalities
from datetime import datetime

#Visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Sentimental Analysis
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('punkt')
nltk.download('vader_lexicon')

#Lenguaje detection 
from langdetect import detect

#Similarity
from sklearn.metrics.pairwise import cosine_similarity


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rrangel4\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\rrangel4\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


**Descompress, Read .json and Unnesting**

In [9]:
#READING (Decompress and read json)  ------- 'steam_games' database
steam_games = '../../NOT_TO_GITHUB/Datasources/steam_games.json.gz'

data_list = []
non_read = []

with gzip.open(steam_games,'rt',encoding='utf-8') as archivo:
    for number_line, line in enumerate(archivo,start=1):
        #print(line)
        try:
            data = json.loads(line)
            data_list.append(data)
            

        except json.JSONDecodeError as e:
            #print(f"Error al decodificar JSON en la línea {number_line}: {e}")
            try:
                data = ast.literal_eval(line) # Mas seguro. manejar expresiones literales simples, como números, cadenas, listas, diccionarios, tuplas y valores booleanos.
                if isinstance(data,dict):
                    data_list.append(data)
                else:
                    non_read.append(number_line)
            except json.JSONDecodeError as e:
                non_read.append(number_line)

steam_games_df = pd.DataFrame(data_list)
steam_games_df.tail(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"
120444,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN


In [5]:
#READING (Decompress and read json) ------- 'user_reviews' database
data_list = []
non_read = []

with gzip.open('../../NOT_TO_GITHUB/Datasources/user_reviews.json.gz','rt',encoding='utf-8') as archivo:
    for number_line, line in enumerate(archivo,start=1):
        #print(line)
        try:
            data = json.loads(line)
            data_list.append(data)
        except json.JSONDecodeError as e:
            #print(f"Error al decodificar JSON en la línea {number_line}: {e}")
            try:
                data = ast.literal_eval(line)
                if isinstance(data,dict):
                    data_list.append(data)
                else:
                    non_read.append(number_line)
            except json.JSONDecodeError as e:
                non_read.append(number_line)

user_reviews_df = pd.DataFrame(data_list)
user_reviews_df.head(2)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."


In [34]:
user_reviews_unnesting_df.to_csv('../../NOT_TO_GITHUB/Datasources/user_reviews_unnesting_df.csv',index=False)

In [21]:
user_reviews_unnesting_df = pd.DataFrame()

for indice, row in user_reviews_df.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    reviews = row['reviews']

    for diccionario in reviews:
        fila = {'user_id':user_id,'user_url':user_url,**diccionario}
        user_reviews_unnesting_df = pd.concat([user_reviews_unnesting_df, pd.DataFrame([fila])], ignore_index=True)

user_reviews_unnesting_df

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


In [ ]:
#READING (Decompress and read json)  ------- 'user_items' database 
data_list = []
non_read = []
conteo_1 = 0 # .json.loads()
conteo_2 = 0 # ast.literal_eval()
conteo_3 = 0 # it Couldn't be do.
conteo_total = conteo_total
with gzip.open('../../NOT_TO_GITHUB/Datasources/users_items.json.gz','rt',encoding='utf-8') as archivo:
    for number_line, line in enumerate(archivo,start=1):
        try:
            data = json.loads(line)
            data_list.append(data)
            conteo_1+=1
            
        except json.JSONDecodeError as e:
            try:
                data = ast.literal_eval(line) 
                if isinstance(data,dict):
                    data_list.append(data)
                    conteo_2+=1
                else:
                    non_read.append(number_line)
                    conteo_3+=1
            except json.JSONDecodeError as e:
                non_read.append(number_line)
                conteo_3+=1
        conteo_4 = conteo_1 + conteo_2 + conteo_3
        if (conteo_4) % 5000 == 0:
            print((conteo_4/conteo_total)*100)

user_items_df = pd.DataFrame(data_list)
user_items_df.head(2)

In [12]:
#UNNESTING  ------- 'user_items' database 

#Take items column and convert each element of list in row
item_column = user_items_df['items'].explode()

# Filter elements that aren't diccionaries
item_column_filter = item_column[item_column.apply(lambda x: isinstance(x, dict))]

# turn this serie into dataframe
user_items_unnesting_df = pd.DataFrame(list(item_column_filter), index=item_column_filter.index)

# merge with original dataframe and reset the index.
user_items_unnesting_df = pd.merge(user_items_df,user_items_unnesting_df, how='left', left_index=True, right_index=True).reset_index()
user_items_unnesting_df.head(2)

,index,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...",10,Counter-Strike,6.0,0.0
1,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...",20,Team Fortress Classic,0.0,0.0


<span style="font-size: 24px; font-weight: bold;">Sentimental score and mood</span>  

In [26]:
#Sentimental analysis of the Reviews column on the user_reviews dataset:
analyzer = SentimentIntensityAnalyzer()
#Add original Score from sentimental analysis (compound)
user_reviews_unnesting_df['sentimental_score'] = user_reviews_unnesting_df['review'].apply(lambda x: list((analyzer.polarity_scores(x)).values())[3])

user_reviews_unnesting_df['mood'] = user_reviews_unnesting_df['review'].apply(lambda x: 2 if (list((analyzer.polarity_scores(x)).values())[3]) >   0.1 else #Positive
                                                                                             ( 1 if (list((analyzer.polarity_scores(x)).values())[3]) >= -0.1 else #Neutral
                                                                                               0 ))                                                                #Negative


Functions
--

**def PlayTimeGenre ( genre : str)**

Return year with most hours played for that genre.

Example return: {"Year of release with most hours played for Genre X". : 2013}

In [35]:
#Transformations to get the genres by title 
#Exploding the steam games dataset by genres
exploded_genres_by_title = steam_games_df[['id','title','genres','release_date']].explode('genres')
#Eliminating null rows
exploded_genres_by_title.drop(exploded_genres_by_title.loc[exploded_genres_by_title['genres'].isna()].index, inplace=True)
exploded_genres_by_title.drop(exploded_genres_by_title.loc[exploded_genres_by_title['release_date'].isna()].index, inplace=True)

In [36]:
#Function to ensure values in column release_date
def valid_date(rdate):
    try:
        pd.to_datetime(rdate)
        return True
    except (TypeError, ValueError):
        return False
    
#Applying valid_date function on release_date field
exploded_genres_by_title['release_date'] = exploded_genres_by_title['release_date'].apply(lambda x: pd.to_datetime(x) if valid_date(x) else None)    

In [37]:
#Extracting year from release_date
exploded_genres_by_title['release_year'] = pd.DatetimeIndex(exploded_genres_by_title['release_date']).year

exploded_genres_by_title .head(5)

,id,title,genres,release_date,release_year
88310,761140,Lost Summoner Kitty,Action,2018-01-04,2018.0
88310,761140,Lost Summoner Kitty,Casual,2018-01-04,2018.0
88310,761140,Lost Summoner Kitty,Indie,2018-01-04,2018.0
88310,761140,Lost Summoner Kitty,Simulation,2018-01-04,2018.0
88310,761140,Lost Summoner Kitty,Strategy,2018-01-04,2018.0


In [38]:
#joining the user_items dataset with the genres_by_title dataset to get the played hours by title and by genre
hours_by_genres_by_title = pd.merge(exploded_genres_by_title,user_items_unnesting_df[['item_id','playtime_forever']], how='inner', left_on = 'id', right_on= 'item_id').reset_index()
hours_by_genres_by_title.head(5)

,index,id,title,genres,release_date,release_year,item_id,playtime_forever
0,0,282010,Carmageddon Max Pack,Action,1997-06-30,1997.0,282010,5.0
1,1,282010,Carmageddon Max Pack,Action,1997-06-30,1997.0,282010,0.0
2,2,282010,Carmageddon Max Pack,Action,1997-06-30,1997.0,282010,0.0
3,3,282010,Carmageddon Max Pack,Action,1997-06-30,1997.0,282010,0.0
4,4,282010,Carmageddon Max Pack,Action,1997-06-30,1997.0,282010,13.0


In [39]:
#Calculating totals by release_year by genre
totplaytime_x_genre = hours_by_genres_by_title[['genres','release_year','playtime_forever']].groupby(['genres','release_year']).sum()
totplaytime_x_genre.reset_index(inplace=True)
totplaytime_x_genre.rename(columns={"playtime_forever": "total_playtime"}, inplace=True)


In [41]:
#Calculating the max playtime by genre
maxplaytime_x_genre = totplaytime_x_genre[['genres','total_playtime']].groupby(['genres']).max()
maxplaytime_x_genre.reset_index(inplace=True)
maxplaytime_x_genre.rename(columns={"total_playtime": "max_playtime"}, inplace=True)
maxplaytime_x_genre.head()


,genres,max_playtime
0,Action,1.099063e+09
1,Adventure,2.243480e+08
2,Animation &amp; Modeling,1.354685e+06
3,Audio Production,4.662460e+05
4,Casual,8.251535e+07


In [43]:
#Identifying the year by genre corresponding to the max total_playtime
maxplaytime_year_by_genre = pd.merge(maxplaytime_x_genre,totplaytime_x_genre,how='left', on = 'genres').reset_index()
maxplaytime_year_by_genre = maxplaytime_year_by_genre[maxplaytime_year_by_genre['total_playtime'] == maxplaytime_year_by_genre['max_playtime']]
maxplaytime_year_by_genre = maxplaytime_year_by_genre[['genres','release_year']]

#Sending the final dataset to a flat file
maxplaytime_year_by_genre.to_csv('../Data/maxplaytime_year_by_genre.csv',',',index=False)

**def UserForGenre(genre : str)**

Return the user who accumulates the most hours played for the given genre and a list of the accumulation of hours played per year.


Example return: {"User with the most hours played for Genre X". : us213ndjss09sdf, "Played Hours":[{Year: 2013, Hours: 203}, {Year: 2012, Hours: 100}, {Year: 2011, Hours: 23}]}

In [44]:
#joining the user_items dataset with the genres_by_title dataset to get the played hours by title and by genre
hours_by_genres_by_user = pd.merge(exploded_genres_by_title[['id','title','genres']],user_items_unnesting_df[['item_id', 'user_id','playtime_forever']], how='inner', left_on = 'id', right_on= 'item_id').reset_index()
hours_by_genres_by_user.head(5)

,index,id,title,genres,item_id,user_id,playtime_forever
0,0,282010,Carmageddon Max Pack,Action,282010,UTNerd24,5.0
1,1,282010,Carmageddon Max Pack,Action,282010,I_DID_911_JUST_SAYING,0.0
2,2,282010,Carmageddon Max Pack,Action,282010,76561197962104795,0.0
3,3,282010,Carmageddon Max Pack,Action,282010,r3ap3r78,0.0
4,4,282010,Carmageddon Max Pack,Action,282010,saint556,13.0


In [45]:
#Calculating total hours by user by genre
totplaytime_x_user_x_genre = hours_by_genres_by_user[['genres','user_id','playtime_forever']].groupby(['genres','user_id']).sum()
totplaytime_x_user_x_genre.reset_index(inplace=True)
totplaytime_x_user_x_genre.rename(columns={"playtime_forever": "total_playtime"}, inplace=True)
totplaytime_x_user_x_genre.head(3)

,genres,user_id,total_playtime
0,Action,--000--,139469.0
1,Action,--ace--,69325.0
2,Action,--ionex--,38315.0


In [46]:
#Calculating maximum playtime by genre
maxplaytime_x_genre = totplaytime_x_user_x_genre[['genres','total_playtime']].groupby(['genres']).max()
maxplaytime_x_genre.rename(columns={"total_playtime": "max_playtime"}, inplace=True)
maxplaytime_x_genre.reset_index(inplace=True)
maxplaytime_x_genre.head(3)

,genres,max_playtime
0,Action,1697650.0
1,Adventure,2183682.0
2,Animation &amp; Modeling,168314.0


In [48]:
#identifying the user whose played hours by genre corresponds with the maximum hours by genre
maxplaytime_by_user_by_genre = pd.merge(maxplaytime_x_genre,totplaytime_x_user_x_genre, how='left', on='genres').reset_index()
maxplaytime_by_user_by_genre = maxplaytime_by_user_by_genre[maxplaytime_by_user_by_genre['total_playtime'] == maxplaytime_by_user_by_genre['max_playtime']]
maxplaytime_by_user_by_genre = maxplaytime_by_user_by_genre[['genres','user_id']]

#Sending the final dataset to a flat file
maxplaytime_by_user_by_genre.to_csv('../Data/maxplaytime_by_user_by_genre.csv',',',index=False)

**def UsersRecommend(year : int)**

Returns the top 3 most recommended games by users for the given year (reviews.recommend = True and positive/neutral reviews).


Example return: [{"Rank 1" : X}, {"Rank 2" : Y},{"Rank 3" : Z}]

In [49]:
#Applying valid_date function on release_date field
steam_games_df['release_date'] = steam_games_df['release_date'].apply(lambda x: pd.to_datetime(x) if valid_date(x) else None)
#Extracting year from release_date
steam_games_df['release_year'] = pd.DatetimeIndex(steam_games_df['release_date']).year

#Joining the steam_games and user_reviews datasets to gather data in one only place
reviews_by_year = pd.merge(steam_games_df[['release_year', 'release_date', 'id', 'app_name', 'title']][steam_games_df.app_name.notna()], user_reviews_unnesting_df[['user_id', 'item_id','posted','recommend','mood']], how = 'inner', left_on = 'id', right_on = 'item_id').reset_index()
reviews_by_year.head(5)

,index,release_year,release_date,id,app_name,title,user_id,item_id,posted,recommend,mood
0,0,1997.0,1997-06-30,282010,Carmageddon Max Pack,Carmageddon Max Pack,InstigatorAU,282010,Posted January 12.,True,1
1,1,1998.0,1998-11-08,70,Half-Life,Half-Life,EizanAratoFujimaki,70,"Posted October 28, 2015.",True,2
2,2,1998.0,1998-11-08,70,Half-Life,Half-Life,GamerFag,70,"Posted January 27, 2011.",True,0
3,3,1998.0,1998-11-08,70,Half-Life,Half-Life,76561198020928326,70,"Posted July 1, 2014.",True,2
4,4,1998.0,1998-11-08,70,Half-Life,Half-Life,Bluegills,70,"Posted December 6, 2013.",True,2


In [53]:
#Function to convert 'posted' column in truly dates
def convert_dates_to_truly_dates(posted_date, rel_year):
    #Using regular expressions to extract month, day and year (if is given) from the 'posted' field
    patron = r"Posted (\w+) (\d+)(?:, (\d{4}))?"
    matches = re.match(patron, posted_date)
    if matches:
        month = matches.group(1)
        posted_day = int(matches.group(2))
        posted_year = int(matches.group(3)) if matches.group(3) is not None else rel_year
        month_number = datetime.strptime(month, '%B').month
        try:
            valid_posted_date = pd.Timestamp(year=posted_year, month=month_number, day=posted_day)
            return valid_posted_date
        except (TypeError, ValueError):
            return None #If conversion to Timestamp is not successfull, None is returned
    else:
        return None  #If posted_date doesn't match with the expected format, None is returned

#Generating posted_date column by converting posted in truly dates
reviews_by_year['posted_date'] = reviews_by_year.apply(lambda row: convert_dates_to_truly_dates(row['posted'], row['release_year']), axis=1)
reviews_by_year.head(3)


,index,release_year,release_date,id,app_name,title,user_id,item_id,posted,recommend,mood,posted_date
0,0,1997.0,1997-06-30,282010,Carmageddon Max Pack,Carmageddon Max Pack,InstigatorAU,282010,Posted January 12.,True,1,NaT
1,1,1998.0,1998-11-08,70,Half-Life,Half-Life,EizanAratoFujimaki,70,"Posted October 28, 2015.",True,2,2015-10-28
2,2,1998.0,1998-11-08,70,Half-Life,Half-Life,GamerFag,70,"Posted January 27, 2011.",True,0,2011-01-27


In [54]:
#Extracting year from posted_date
reviews_by_year['posted_year'] = pd.DatetimeIndex(reviews_by_year['posted_date']).year

reviews_by_year .head(5)

,index,release_year,release_date,id,app_name,title,user_id,item_id,posted,recommend,mood,posted_date,posted_year
0,0,1997.0,1997-06-30,282010,Carmageddon Max Pack,Carmageddon Max Pack,InstigatorAU,282010,Posted January 12.,True,1,NaT,NaN
1,1,1998.0,1998-11-08,70,Half-Life,Half-Life,EizanAratoFujimaki,70,"Posted October 28, 2015.",True,2,2015-10-28,2015.0
2,2,1998.0,1998-11-08,70,Half-Life,Half-Life,GamerFag,70,"Posted January 27, 2011.",True,0,2011-01-27,2011.0
3,3,1998.0,1998-11-08,70,Half-Life,Half-Life,76561198020928326,70,"Posted July 1, 2014.",True,2,2014-07-01,2014.0
4,4,1998.0,1998-11-08,70,Half-Life,Half-Life,Bluegills,70,"Posted December 6, 2013.",True,2,2013-12-06,2013.0


In [56]:
#Identifying whether the user truly recommends the title or not
def more_recomended (recommended , review_sentiment):
    return 1 if ((recommended ==True) and ((review_sentiment == 1) or (review_sentiment == 2))) else 0

#Applying the function "more_recomended" to generate the column more_recommended in the reviews_by_year dataset
reviews_by_year['more_recommended'] = reviews_by_year.apply(lambda row: more_recomended(row['recommend'], row['mood']), axis=1)

reviews_by_year.head()

,index,release_year,release_date,id,app_name,title,user_id,item_id,posted,recommend,mood,posted_date,posted_year,more_recommended
0,0,1997.0,1997-06-30,282010,Carmageddon Max Pack,Carmageddon Max Pack,InstigatorAU,282010,Posted January 12.,True,1,NaT,NaN,1
1,1,1998.0,1998-11-08,70,Half-Life,Half-Life,EizanAratoFujimaki,70,"Posted October 28, 2015.",True,2,2015-10-28,2015.0,1
2,2,1998.0,1998-11-08,70,Half-Life,Half-Life,GamerFag,70,"Posted January 27, 2011.",True,0,2011-01-27,2011.0,0
3,3,1998.0,1998-11-08,70,Half-Life,Half-Life,76561198020928326,70,"Posted July 1, 2014.",True,2,2014-07-01,2014.0,1
4,4,1998.0,1998-11-08,70,Half-Life,Half-Life,Bluegills,70,"Posted December 6, 2013.",True,2,2013-12-06,2013.0,1


In [57]:
#Counting recommended times by game in each year
games_recommended_by_year = reviews_by_year[['id','title','posted_year','more_recommended']].groupby(['id','title','posted_year']).sum()
games_recommended_by_year.reset_index(inplace=True)
games_recommended_by_year.rename(columns={"more_recommended": "times_recommended"}, inplace=True)
games_recommended_by_year.sort_values(by = ['posted_year','times_recommended'], ascending=[True,False])

,id,title,posted_year,times_recommended
3668,440,Team Fortress 2,2010.0,8
202,1250,Killing Floor,2010.0,5
3925,630,Alien Swarm,2010.0,4
3523,4000,Garry's Mod,2010.0,3
1139,22600,Worms Reloaded,2010.0,2
...,...,...,...,...
4169,9350,Supreme Commander,2015.0,0
4173,94000,Dinner Date,2015.0,0
4187,94590,Puzzle Agent 2,2015.0,0
4208,9730,Tycoon City: New York,2015.0,0


In [58]:
#Sending the final dataset to a flat file
games_recommended_by_year.to_csv('../Data/games_recommended_by_year.csv',',',index=False)

**def UsersNotRecommend(year : int)**

Returns the top 3 least recommended games by users for the given year (reviews.recommend = False and negative reviews).


Example return: [{"Rank 1" : X}, {"Rank 2" : Y},{"Rank 3" : Z}]

In [61]:
#Identifying whether the user truly discourage the title or not
def not_recomended (recommended , review_sentiment):
    return 1 if ((recommended ==False) and ((review_sentiment == 0))) else 0

worse_reviews_by_year = reviews_by_year.copy()
worse_reviews_by_year = worse_reviews_by_year.drop(columns = ['more_recommended'])
worse_reviews_by_year['not_recommended'] = worse_reviews_by_year.apply(lambda row: not_recomended(row['recommend'], row['mood']), axis=1)

worse_reviews_by_year.sort_values(by = ['not_recommended'], ascending=False)
worse_reviews_by_year.head()

,index,release_year,release_date,id,app_name,title,user_id,item_id,posted,recommend,mood,posted_date,posted_year,not_recommended
0,0,1997.0,1997-06-30,282010,Carmageddon Max Pack,Carmageddon Max Pack,InstigatorAU,282010,Posted January 12.,True,1,NaT,NaN,0
1,1,1998.0,1998-11-08,70,Half-Life,Half-Life,EizanAratoFujimaki,70,"Posted October 28, 2015.",True,2,2015-10-28,2015.0,0
2,2,1998.0,1998-11-08,70,Half-Life,Half-Life,GamerFag,70,"Posted January 27, 2011.",True,0,2011-01-27,2011.0,0
3,3,1998.0,1998-11-08,70,Half-Life,Half-Life,76561198020928326,70,"Posted July 1, 2014.",True,2,2014-07-01,2014.0,0
4,4,1998.0,1998-11-08,70,Half-Life,Half-Life,Bluegills,70,"Posted December 6, 2013.",True,2,2013-12-06,2013.0,0


In [63]:
#Counting not recommended times by game in each year
games_not_recommended_by_year =  worse_reviews_by_year[['title','posted_year','not_recommended']][worse_reviews_by_year.not_recommended == 1].groupby(['title','posted_year']).sum().sort_values(by = ['posted_year','not_recommended'], ascending = [True,False])
games_not_recommended_by_year.reset_index(inplace=True)
games_not_recommended_by_year.rename(columns={"not_recommended": "times_not_recommended"}, inplace=True)
games_not_recommended_by_year.head()


,title,posted_year,times_not_recommended
0,And Yet It Moves,2011.0,2
1,Men of War: Vietnam,2011.0,1
2,Team Fortress 2,2011.0,1
3,Call of Duty®: Black Ops II,2012.0,1
4,Red Faction®: Armageddon™,2012.0,1


In [64]:
#Sending the final dataset to a flat file
games_not_recommended_by_year.to_csv('../Data/games_not_recommended_by_year.csv',',',index=False)

**def sentiment_analysis( year : int )**

Based on the year of release, a list is returned with the number of user review records that are categorized with a sentiment analysis.

**Example return:** {Negative = 182, Neutral = 120, Positive = 278}

In [66]:
# Created a dataset with columns that it need from reviews
user_reviews_unnesting_df_2 = user_reviews_unnesting_df[['item_id', 'mood']].copy()
print(user_reviews_unnesting_df_2.shape)
user_reviews_unnesting_df_2.head()

(59305, 2)


,item_id,mood
0,1250,2
1,22200,2
2,43110,2
3,251610,2
4,227300,2


In [67]:
# How many 'NaN' columns there are in each row
steam_games_df.isna().sum(axis=1).value_counts()

14    88310
0     22528
1      5899
6      1935
3       912
4       397
2       337
7       117
5         9
11        1
dtype: int64

In [68]:
#Removed rows where all their columns (13) are 'NaN'
steam_games_df_2 = steam_games_df.copy()
steam_games_df_2['Conteo_NaN'] = steam_games_df_2.isna().sum(axis=1)
steam_games_df_2 = steam_games_df_2[steam_games_df_2['Conteo_NaN']!=13][['id', 'release_date']]

#Remove all rows that have nan in 'release_date' column
steam_games_df_2.dropna(subset=['release_date'],inplace=True)


In [70]:

#Merge Release_date of game with mood from reviews
reviews_release_date = (pd.merge(steam_games_df_2, user_reviews_unnesting_df_2, left_on='id', right_on='item_id', how='right'))[['release_date', 'id', 'mood']]

# Add column with date Format:
date = []
for i in reviews_release_date['release_date']:
    date.append(pd.to_datetime(i))

reviews_release_date['date'] = date

#Add year column and remove 'nan' of year column
reviews_release_date['year'] = reviews_release_date['date'].dt.year
reviews_release_date.dropna(subset=['year'], inplace=True)


In [72]:
# Logical of function to generates
def sentiment_analysis(year):
    #year=2015
    pivot = (reviews_release_date[reviews_release_date['year']==year].pivot_table(index='mood',aggfunc='size')).to_frame()
    try:
        negative = pivot[pivot.index==0].to_numpy().tolist()[0][0]
    except Exception as e:
        negative = 0

    try:
        neutral = pivot[pivot.index==1].to_numpy().tolist()[0][0]
    except Exception as e:
        neutral = 0
    
    try:
        positive = pivot[pivot.index==2].to_numpy().tolist()[0][0]
    except Exception as e:
        positive = 0

    scores = f"Negative = {negative}, Neutral = {neutral} and Positive = {positive}"
    return scores
#Test
sentiment_analysis(1995)

'Negative = 5, Neutral = 0 and Positive = 11'

In [75]:
#Sending the final dataset to a flat file
sentiment_analysis_df = pd.DataFrame(reviews_release_date['year'].unique()).rename(columns={0:'year'})
sentiment_analysis_df['sentiment'] = sentiment_analysis_df['year'].apply(lambda x : sentiment_analysis(x))
sentiment_analysis_df.to_csv('../Data\sentiment_analysis_df.csv',index=False)


In [78]:
sentiment_analysis_df.head()

,year,sentiment
0,2009.0,"Negative = 312, Neutral = 478 and Positive = 1186"
1,2013.0,"Negative = 1213, Neutral = 1548 and Positive =..."
2,2015.0,"Negative = 1213, Neutral = 1417 and Positive =..."
3,2014.0,"Negative = 1167, Neutral = 1298 and Positive =..."
4,2011.0,"Negative = 456, Neutral = 760 and Positive = 2238"


<span style="font-size: 24px; font-weight: bold;">ML Model</span>  

In [79]:
# Created pivot of sentimental analysis (0,1 or 2) between user_id and item_id
pivot_user_item_review = pd.pivot_table(user_reviews_unnesting_df, index='user_id', columns='item_id', values='mood', aggfunc='mean', fill_value=0)
pivot_user_item_review

item_id,10,10090,10130,10140,10150,10180,10220,102500,102600,102700,...,9900,99100,9930,99300,9940,99400,99700,99810,99900,99910
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
--ace--,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
--ionex--,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-2SV-vuLB-Kg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-Azsael-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zwanzigdrei,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zy0705,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zynxgameth,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
#Created similarity array from sentimental analysis (0,1 or 2) between user_id and item_id
similarity_pivot = cosine_similarity(pivot_user_item_review)
print(similarity_pivot)

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.         0.14142136 0.        ]
 [0.         0.         1.         ... 0.         0.28284271 0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.14142136 0.28284271 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


**Dataset to API reading**

In [84]:
user_index = pivot_user_item_review.index # listado de usuarios
user_id_reco = pivot_user_item_review.columns # listado Items
Id_Name = steam_games_df[(~steam_games_df['title'].isna()) & (~steam_games_df[['title','id']].duplicated())][['title','id']]

def recomendacion_usuario(Id_de_Usuario):
    
    userID_position = user_index.get_loc(Id_de_Usuario)
    similarity_pivot_user = similarity_pivot[userID_position]
    
    weighted_of_items = similarity_pivot_user.dot(pivot_user_item_review.values) / np.abs(similarity_pivot_user).sum()
    weighted_of_items_index_V2 = np.argsort(weighted_of_items)[-5:][::-1]

    Top_games = pd.DataFrame(user_id_reco[weighted_of_items_index_V2].to_numpy()) # top 5 en Id_items
    Top_games = pd.merge(Top_games,Id_Name, left_on=0, right_on='id')['title'].to_list()

    return Top_games

#Test function
userID = '76561198094665607'
recomendacion_usuario(userID)

["Garry's Mod",
 'Killing Floor',
 'Team Fortress 2',
 'Counter-Strike: Global Offensive',
 'Fallout 4']

In [ ]:
#Creation of database of Games recomendation by user_id

database_recomendations = pd.DataFrame(user_reviews_unnesting_df['user_id'].unique()).rename(columns={0:'user_id'})
database_recomendations['recomendations'] = database_recomendations['user_id'].progress_apply(lambda x : recomendacion_usuario(x))
database_recomendations.to_csv('Render\database_recomendations.csv',index=False)

**Testing for the  FastApi functions**

In [86]:
#Loanding flat files to be used by the functions
maxplaytime_year_by_genre = pd.read_csv("../Data/maxplaytime_year_by_genre.csv")

    

def PlayTimeGenre(genre: str):
    '''Return the the release year with more played hours for the entered genre'''
    # Get the corresponding year based on the gender
    year = maxplaytime_year_by_genre.loc[maxplaytime_year_by_genre['genres'].str.lower() == genre.lower(), 'release_year'].values
    if len(year) == 0:
        return {f'Videogame release year for the genre {genre}': 'no encontrado'}
    else:
        return {f'Videogame release year for the genre {genre}': str(year[0])}
    

PlayTimeGenre('Action')    

{'Videogame release year for the genre Action': '2012.0'}

In [87]:
#Loanding flat files to be used by the functions
maxplaytime_by_user_by_genre = pd.read_csv("../Data/maxplaytime_by_user_by_genre.csv")

    

def UserForGenre(genre: str):
    '''Return the user who accumulates the most hours played for the given genre and a list of the accumulation of hours played per year.'''
    # Get the corresponding user based on the gender
    user = maxplaytime_by_user_by_genre.loc[maxplaytime_by_user_by_genre['genres'].str.lower() == genre.lower(), 'user_id'].values
    if len(user) == 0:
        return {f'User with more hours played for Gen  {genre}': 'no encontrado'}
    else:
        return {f'User with more hours played for Gen  {genre}': str(user[0])}
    

UserForGenre('Adventure')    

{'User with more hours played for Gen  Adventure': 'REBAS_AS_F-T'}

In [88]:
#Loanding flat files to be used by the functions
games_recommended_by_year = pd.read_csv("../Data/games_recommended_by_year.csv")

def UsersRecommend( year : int ): 
    '''Returns the top 3 recommended games by users for the given year (reviews.recommend = True and positive/neutral reviews).'''
    #Filtering by year
    filtered_df = games_recommended_by_year[games_recommended_by_year['posted_year'] == year]
    #Sorting by  'times_recommended' in a descending order
    sorted_df = filtered_df.sort_values(by='times_recommended', ascending=False)
    #Selecting the 3 first records
    top_3_games = sorted_df.head(3)    
    lista_de_diccionarios = top_3_games[['title']].to_dict(orient='records')
    return lista_de_diccionarios


UsersRecommend( 2010)

[{'title': 'Team Fortress 2'},
 {'title': 'Killing Floor'},
 {'title': 'Alien Swarm'}]

In [91]:
#Loanding flat files to be used by the functions
more_not_recommended_by_users = pd.read_csv("../Data/games_not_recommended_by_year.csv")

def UsersNotRecommend( year : int ): 
    '''Returns the top 3 recommended games by users for the given year (reviews.recommend = True and positive/neutral reviews).'''
    #Filtering by year
    filtered_df = more_not_recommended_by_users[more_not_recommended_by_users['posted_year'] == year]
    #Sorting by  'times_recommended' in a descending order
    sorted_df = filtered_df.sort_values(by='times_not_recommended', ascending=False)
    #Selecting the 3 first records
    top_3_games = sorted_df.head(3)    
    lista_de_diccionarios = top_3_games[['title']].to_dict(orient='records')
    if len(lista_de_diccionarios) == 0:
        return []
    else:
        return lista_de_diccionarios


UsersNotRecommend( 2011)

[{'title': 'And Yet It Moves'},
 {'title': 'Men of War: Vietnam'},
 {'title': 'Team Fortress 2'}]